In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I.

In [2]:
diccionario = {'usa': [39.7837304, -100.445882], "australia" : [-24.7761086, 134.755],
        'south africa' : [-28.8166236, 24.991639], 'new zealand': [-41.5000831, 172.8344077], 'papua new guinea': [-5.6816069, 144.2489081]}


lista_paises = ['usa','australia','new zealand','south africa','papua new guinea']

In [3]:
class extraer():
    def __init__(self, diccionario):
        self.diccionario = diccionario 
        self.paises = lista_paises
    
    def cargar_dfs():
        df_attacks = pd.read_csv("attacks_limpieza_completa.csv", index_col = 0)
        lista_paises = ['usa','australia','new zealand','south africa','papua new guinea']
        df_paises = df_attacks[df_attacks['country'].isin(lista_paises)]
        df_clima = pd.read_csv("datos_clima.csv", index_col = 0)

      
    def llamar_API(diccionario):
        
    #Hacemos un for para ir iterando por los paises y sus valores e ir extrayendo la info de la API
        for key, value in diccionario.items():
            producto = 'meteo'
            lat = value[0]
            lon = value[1]
            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
            response = requests.get(url=url)
            response.status_code
            response.reason
            df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
            #Creamos una columna con el nombre del pais
            df['pais'] = key
            return df

    # esto está repetido arriba...
    # aquí estaba cargar_dfs() repetido...

    #Concatenamos los archivos, creamos un df vacio para ir rellenandolo con la info que queremos
    def juntar_dfs(self, df_completo, df_civil, df_visibilidad, df_meteo): 
    
        # lo primero que hacemos es concatenar los dataframes con la información general. df_completo y df_civil
        df_completo = pd.concat([df_completo, df_civil], axis = 0)

        # ahora es el turno de unir el dataframe con la información de la visibilidad con el completo
        # en este caso el how lo ponemos como left ya que queremos que se quede con toda la info que en la primera tabla que le pasamos que es la que tiene toda la información
        df_completo = pd.merge(df_completo , df_visibilidad , on=['fecha', "timepoint"], how = "inner")
        df_completo = pd.merge(df_completo, df_meteo, on=['fecha'], how = "left")
        # guardamos los datos
        df_completo.to_pickle('./datos_actualizados.pkl')
        df_completo.to_csv('./datos_actualizados.csv')

        return df_completo

    def limpiar_dataframe(self, df, lista_columnas): 

       #convertimos la fecha a datetime
        df["fecha"] = pd.to_datetime(df["fecha"])

        # reemplazamos los nulos de las columnas por la media
         # lista de columnas en las que queremos reemplazar los nulos
        df[lista_columnas]=df[lista_columnas].fillna(df.mean())

        # quitar columnas repetidas:
        
        df.drop(["ciudad_y"], axis = 1, inplace = True)
        
        # renombrar columnas
        
        df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

        # quitar % 
        df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

        # guardamos los datos una vez limpios
        df.to_pickle('./datos_Madrid.pkl')
        df.to_csv('./datos_Madrid.csv')

        return df